In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from mdaq import pymdaq_web
import datetime
import time
import pandas as pd
import sys
import codecs
import os
import sys
import binascii
import requests
from requests.exceptions import Timeout
import pandas as pd
import openpyxl
import subprocess
import copy
import libCom
import schedule

In [2]:
db = pymdaq_web.db("xfweb-dmz-03", port=8888)

In [3]:
################################リスト操作の関数################################

In [4]:
#辞書型のリストから指定したkeyと一致（一致 flg = 0, 不一致　flg = 0以外）するリストを返す
def extract_list_specified_key(list, key, find_value, flg):
    list_tmp = []
        
    for i in range(len(list)):
        if flg == 0:
            if list[i][key] == find_value:
                list_tmp.append(list[i]) 
        else:
            if list[i][key] != find_value:
                list_tmp.append(list[i]) 

    return list_tmp

In [5]:
################################GUN HV OFF 時間の取得################################

In [6]:
#期間内のGUN ON/OFF時間を取得する。最大31日未満。
def get_gun_status_long_time(time_beg, time_end):
    limit_day = 400
    
    
    time_list = []
    hv_list = []
    
    if (time_end-time_beg).days >= limit_day or (time_end-time_beg).days < 0:
        ret = -1
        print("DB GET Time ERROR")
    else:
        time_bk = time_beg
        while (time_bk - time_end).days < 0:
            time_list.append(time_bk.strftime("%Y/%m/%d %H:%M:%S"))
            time_bk = time_bk + datetime.timedelta(days = 2)

        time_list.append(time_end.strftime("%Y/%m/%d %H:%M:%S"))
        # print(timeList)
        for i in range(len(time_list)):
            if i < (len(time_list)-1):
                res = get_gun_status_db(time_list[i], time_list[i+1])
                if type(res) is list:
                    hv_list.extend(res)
                time.sleep(1)       
        return hv_list
    


#範囲時間内のGUNのステータスを取得
def get_gun_status_db(dt_beg, dt_end):
    hv_list =[]
    signal_name = 'scss_tmg_gun_tdu/status'
    
    ret = check_db_get_time(dt_beg, dt_end)
    
    if ret == 0:   
        res = db.get_data(signal_name,dt_beg,dt_end)
        if db.status()==pymdaq_web.DB_OK:
            res.sort()
            hv_list = get_gun_off_time(res, dt_end)
            return hv_list
        else:
            print("ERR: %d  Msg: %s"%(db.status(),db.err_msg()))
            return db.status()

            
#DB取得時間が3日未満か確認する
def check_db_get_time(dt_beg, dt_end):

    limit_day =  3
    time = datetime.datetime.strptime(dt_beg, "%Y/%m/%d %H:%M:%S")
    time2 = datetime.datetime.strptime(dt_end, "%Y/%m/%d %H:%M:%S")
    
    if (time2-time).days >= limit_day or (time2-time).days < 0:
        ret = -1
        print("DB GET Time ERROR")
    else:
        ret = 0
    
    return ret

#GUNのステータスリストからGUN ON/OFF時間のリストを取得する

def get_gun_off_time(sig_res, dt_end):
    sig_res.sort()
    list = []
    status = "ON"

    for i in range(len(sig_res)):
        if sig_res[i][1] & (1 << 12) != 0:
            if status == "ON":
                off_time = datetime.datetime.strptime(sig_res[i][0], "%Y/%m/%d %H:%M:%S.%f").replace(microsecond = 0)
                status = "OFF"
        else:
            if status == "OFF":
                on_time = datetime.datetime.strptime(sig_res[i][0], "%Y/%m/%d %H:%M:%S.%f").replace(microsecond = 0)
                status = "ON"
                
                td = on_time - off_time
                if td.total_seconds() >2:#2秒以下を排除
                    list.append([off_time, on_time])

        if (status == "OFF") & (i == (len(sig_res) -1)):
            on_time = datetime.datetime.strptime(dt_end, "%Y/%m/%d %H:%M:%S")
            list.append([off_time, on_time])
        
    
    
    return list

In [7]:
#EXCELにGUN HV OFF時間を書き込む
def write_excel_gun_hv_time(list):
    wb = openpyxl.load_workbook(r"C:\Users\kokubu\Documents\python\OperationSummary\SCSS_計画時間.xlsx")    
    ws = wb.create_sheet()
    ws.title = "test"
    sheet_name = "Chopper OFF"
    offset = 0
    
    for ws2 in wb.worksheets:
        if ws2.title.endswith(sheet_name):
            wb.remove(ws2)
            
    write_excel_gun_hv_time_bl(ws, list)
    libCom.auto_sheet_width(ws)
    ws.title = sheet_name    
    wb.save(r"C:\Users\kokubu\Documents\python\OperationSummary\SCSS_計画時間.xlsx")
    
    
#EXCELにBLごとのGUN HV OFF時間を書き込む
def write_excel_gun_hv_time_bl(ws, list):

              
    ws.cell(1,1 ,value = "SCSS")
    ws.cell(2,1,value = 'Chopper OFF')
    ws.cell(2,2,value = 'chopper ON')


    #配列ループ
    for i in range(0,len(list)):
        #列にリストを書き込み
        ws.cell(i+3,1,value = list[i][0]).number_format = 'yyyy/mm/dd hh:mm'
        ws.cell(i+3,2,value = list[i][1]).number_format = 'yyyy/mm/dd hh:mm'


In [8]:
#指定したformat形式の時間の差分を返す
def calc_delta_time_format(start_time, end_time, format):
    stime = datetime.datetime.strptime(start_time, format)
    etime = datetime.datetime.strptime(end_time, format)
    dtime = etime- stime
    return dtime

#deltatime形式を[hh:mm:ss]の文字列形式に変換する
def format_timedelta(timedelta):
  total_sec = timedelta.total_seconds()
  # hours
  hours = total_sec // 3600 
  # remaining seconds
  remain = total_sec - (hours * 3600)
  # minutes
  minutes = remain // 60
  # remaining seconds
  seconds = remain - (minutes * 60)

  # total time
  return '{:02}:{:02}:{:02}'.format(int(hours), int(minutes), int(seconds))

In [9]:
#日時を指定してGUNのOFF/ON時間の出力する
def output_excel_gun_hvoff_time():
     
    val = input("開始日時を入力してください。　(例)2021/11/1 10:00 >>>")
    try:
        dt_beg = datetime.datetime.strptime(val, "%Y/%m/%d %H:%M")
    except ValueError:
        print ("エラー：日時のフォーマットが正しくありません。")
        return -1

    val = input("終了日時を入力してください　(例)2021/11/15 10:00 >>>")
    try:
        dt_end = datetime.datetime.strptime(val, "%Y/%m/%d %H:%M")
    except ValueError:
        print ("エラー：日時のフォーマットが正しくありません。")
        return -1
    
    
    gun_list = get_gun_status_long_time(dt_beg, dt_end)
    libCom.print_list(gun_list)

    write_excel_gun_hv_time(gun_list)
    return 0

In [10]:
ret = output_excel_gun_hvoff_time()
if ret == 0:
    print("正常終了しました")
    EXCEL = r"C:\Users\kokubu\Documents\python\OperationSummary\SCSS_計画時間.xlsx"
    subprocess.Popen(['start', EXCEL], shell=True)
else:
    print("異常終了しました")
time.sleep(60)
          


開始日時を入力してください。　(例)2021/11/1 10:00 >>> 2021/4/1 00:00
終了日時を入力してください　(例)2021/11/15 10:00 >>> 2022/4/1 00:00


ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -307000003  Msg: No data.
ERR: -30